# KDD 2009 - Tratamento de dados simplista
Como parte do exercício de resolução do KDD Cup de 2009 criamos este notebook para realizar um teste do modelo preditivo sem fazer grandes alterações nos dados originais. A análise completa pode ser vista [aqui](./KDD2009.ipynb).

O objetivo é ter um parâmetro de comparação com o modelo entregue, que utiliza o **vtreat** para gerar um plano de tratamento das variáveis do KDD Cup 2009, para cada uma dos resultados desejados (*appetency*, *churn* e *up-selling*).

Os únicos tratamentos serão:
- Remoção das variáveis nulas (sem nenhum valor válido)
- Conversão das variáveis categóricas para valores numéricos

Os valores NA do conjunto de dados serão mantidos como estão.

In [1]:
# Importação de bibliotecas
import matplotlib as mpl
import numpy as np
import os
import pandas as pd

# Lê arquivo de treinamento

In [2]:
# Diretório de dados
data_folder = os.path.join("..", "data")
# Arquivo de dados
training_data_file = os.path.join(data_folder, "orange_small_train.data")

# Dataframae com os dados de treinamento
kdd2009 = pd.read_csv(training_data_file, delimiter='\t', na_values=[''])

## Cria dataframe de contagem de NaN e remove colunas que só contém NaN

In [3]:
# Cria dataframe com contagem de NaNs
nans = pd.DataFrame(kdd2009.isna().sum(), columns=['NAN_count'])
variaveisNulas = nans[nans['NAN_count'] == 50000].index.values

# Remove as variáveis que só contém NaNs
kdd2009.drop(variaveisNulas, axis=1, inplace=True)

# Lê arquivos de labels
Lê arquivos com os rótulos de *Appetency, Churn* e *Upselling*. Inclui labels no dataframe do kdd2009

In [4]:
# Arquivos com os rótulos das métricas
appetency_labels = os.path.join(data_folder, "orange_small_train_appetency.labels")
churn_labels = os.path.join(data_folder, "orange_small_train_churn.labels" )
upselling_labels = os.path.join(data_folder, "orange_small_train_upselling.labels")

# Leitura dos rótulos dos dados
appetency = pd.read_csv(appetency_labels, header=None, names=["label"])
churn = pd.read_csv(churn_labels, header=None, names=["label"])
upselling = pd.read_csv(upselling_labels, header=None, names=["label"])

# Inclusão dos rótulos no dataframe como colunas adicionais
# Com valores em booleanos:
# positivo (1): TRUE
# negativo (-1): FALSE

# Convertendo os labels (1/-1) para (TRUE/FALSE) - Assume que não há erro nos dados de entrada
appetency['label'] = appetency['label'] == 1
churn['label'] = churn['label'] == 1
upselling['label'] = upselling['label'] == 1

# Colunas de variáveis de entrada
variaveis = kdd2009.keys()

# Incluindo as colunas no dataframe
kdd2009['appetency'] = appetency['label']
kdd2009['churn'] = churn['label']
kdd2009['upselling'] = upselling['label']

# Colunas de resultados
resultados = ['appetency', 'churn', 'upselling']

# Verificando os resultados

In [5]:
# Primeiros valores dos resultados no dataframe de treinamento
kdd2009[resultados].head()

,appetency,churn,upselling
0,False,False,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


## E as variáveis (já sem colunas só de NaNs)

In [6]:
# Primeiros valores das variaveis no datarame de treinamento
kdd2009[variaveis].head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


# Classificação de colunas
Classifica as colunas em inteiros, reais, objetos e booleanos

In [7]:
# Verifica os tipos de dados existentes no dataframe
print('Lista de tipos de dados no kdd2009: {}'.format(list(set(kdd2009.dtypes))))

Lista de tipos de dados no kdd2009: [dtype('bool'), dtype('float64'), dtype('int64'), dtype('O')]


In [8]:
data_types = pd.DataFrame(kdd2009[variaveis].dtypes, columns=['dtype'])

variaveisInteiras = list(data_types[data_types.values == np.dtype('int64')].index.values)
variaveisReais = list(data_types[data_types.values == np.dtype('float64')].index.values)
variaveisCategoricas = list(data_types[data_types.values == np.dtype('object')].index.values)

len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) == len(kdd2009.keys())

print('Há {} variáveis inteiras, {} variáveis reais e {} objetos'.format(
                                                                    len(variaveisInteiras),
                                                                    len(variaveisReais),
                                                                    len(variaveisCategoricas)))

# Verifica se o total de tipos de variáveis está correto
if len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) != len(kdd2009.keys()):
    print('Aviso: erro na contagem de tipos de variáveis!!!')

Há 1 variáveis inteiras, 173 variáveis reais e 38 objetos


In [10]:
kdd2009[variaveisCategoricas].head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,CEat0G8rTN,RO12,NaN,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,jg69tYsGvO,RO12,NaN,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


In [13]:
kdd2009.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,702.000000,1.487000e+03,1240.000000,...,7.020000e+02,44991.000000,1.579000e+03,1.241000e+03,1241.000000,702.000000,702.00000,1241.000000,21022.000000,333.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,...,3.776755e+06,0.611456,1.416638e+06,7.777380e+04,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,154.777855,9.280896e+05,2.869558,...,3.785696e+06,2.495681,2.279786e+06,2.016188e+05,46.973777,8.781967,60.22303,113.980072,86.707692,29085.146490
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,-6.420000,6.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,4.000000,0.000000e+00,8.000000,...,1.917352e+05,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,19.380000,204.000000,2732.670000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,20.000000,0.000000e+00,8.000000,...,2.431310e+06,0.000000,1.167780e+05,0.000000e+00,0.000000,0.000000,4.00000,197.640000,270.000000,12668.940000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,46.000000,2.628630e+05,8.000000,...,6.471827e+06,0.000000,1.844952e+06,4.881000e+04,8.000000,6.000000,14.00000,252.960000,330.000000,29396.340000
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,...,1.428483e+07,49.000000,1.199478e+07,3.048400e+06,1200.000000,102.000000,910.00000,628.620000,642.000000,230427.000000


In [19]:
for col in variaveisCategoricas:  # Iterate over chosen columns
    kdd2009[col] = kdd2009[col].astype(dtype=np.float64)

ValueError: could not convert string to float: 'r__I'

In [18]:
kdd2009.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,False,False,False
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,False,True,False
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,False,False,False
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,False,False,False
